# 4. Predict

Import all you need! 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np 
import pickle
from keras.applications import VGG16
from keras.models import Model, load_model
from keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split

Load pre-processed files (images, probability maps and keypoints): 

In [ ]:
with open('pre_processed_data/X_train_preprocessed.pickle', 'rb') as f:
        X_train_1 = pickle.load(f)
    
with open('pre_processed_data/y_train_preprocessed.pickle', 'rb') as f:
        y_keypoints_train_1 = pickle.load(f)

with open('pre_processed_data/heatmap_train.pickle', 'rb') as f:
        y_train_1 = pickle.load(f)

Division in train and validation as done in training, i.e. with the same seed and the same sizes:

In [ ]:
X_train, X_validation, y_train, y_validation = train_test_split(X_train_1, y_train_1, test_size=0.2, random_state=42)
X_train, X_validation, y_keypoints_train, y_keypoints_validation = train_test_split(X_train_1, y_keypoints_train_1, \
                                                                                    test_size=0.2, random_state=42)

All training data: 

In [ ]:
X_train_1_processed = np.array(X_train_1, dtype = float)

Data divided in train and validation sets:

In [ ]:
X_train_processed = np.array(X_train, dtype=np.float32)
X_validation_processed = np.array(X_validation, dtype=np.float32)

Load model trained:

In [ ]:
model = load_model('models/keypoint_detection.hdf5')

Before predictions, it is necessary to pre-process the data in the same way as did in the training process:

In [ ]:
from keras.applications.vgg16 import preprocess_input
X_train_1_processed = preprocess_input(X_train_1_processed)
X_train_processed = preprocess_input(X_train_processed)
X_validation_processed = preprocess_input(X_validation_processed)

Predictions (probability maps + coordinates of keypoints):

In [ ]:
predictions_train = model.predict(X_train_1_processed, batch_size=1)

Predicted probability maps:

In [ ]:
prob_maps_train = np.array(predictions_train[0])
prob_maps_train = np.reshape(prob_maps_train,(-1,384,512))

Predicted keypoints:

In [ ]:
keypoints_train = np.array(predictions_train[1])

Show predicted probability maps:

In [ ]:
#X_train_1 = np.array(X_train_1)
#for i in range(X_train_1.shape[0]):
#    plt.imshow(X_train_1[i])
#    plt.imshow(prob_maps_train[i], interpolation = 'nearest', alpha = 0.6)
#    plt.show()

To visualize the keypoints, it is necessary to multiply by the value we used to normalize the data

In [ ]:
keypoints_train = keypoints_train * 512

Show predicted keypoints:

In [ ]:
#x = []
#y = [] 

#for i in range(X_train_1.shape[0]):
#    x.append(keypoints_train[i][0:74:2])
#    y.append(keypoints_train[i][1:75:2])     
#    plt.imshow(X_train_1[i], cmap='gray')
#    plt.plot(x[i],y[i],'rx')
#    plt.show()

Save results:

In [ ]:
with open('results/keypoints_train.pickle', 'wb') as f:
    pickle.dump(keypoints_train, f)